In [ ]:
''' Линейная интерполяция'''
def interpolate_sales_by_month(test_declarations):
    results = []

    # Преобразуем столбцы с датами в формат datetime
    test_declarations['Дата начала интервала'] = pd.to_datetime(test_declarations['Дата начала интервала'])
    test_declarations['Дата завершения интервала'] = pd.to_datetime(test_declarations['Дата завершения интервала'])

    # Проходим по каждому ЖК
    for _, row in test_declarations.iterrows():
        start_date = row['Дата начала интервала']
        end_date = row['Дата завершения интервала']
        total_sales = row['Продано квартир (кв. м.)']
        name = row['ЖК']
        region = row['Регион']
        okrug = row['Округ Направление']
        developer = row['Застройщик']
        corpus = row['Корпуса']
        ID = row['ID ЖК']
        price = row['Цена кв. м квартир (руб.)']

        # Рассчитываем продажи за день
        total_days = (end_date - start_date).days + 1
        sales_per_day = total_sales / total_days if total_days > 0 else 0

        # Создаем список месяцев в интервале
        months = pd.date_range(start_date, end_date, freq='MS')

        # Если интервал в пределах одного месяца или меньше
        if len(months) == 0 or (start_date.year == end_date.year and start_date.month == end_date.month):
            results.append({
                'project': name,
                'year': start_date.year,
                'months': start_date.month,
                'sales_live': total_sales,
                'developer': developer,
                'region': region,
                'okrug': okrug,
                'corpus': corpus,
                'ID': ID,
                'price':price
            })
            continue

        # Обрабатываем каждый месяц
        for i, month_start in enumerate(months):
            # Определяем начало и конец периода для текущего месяца
            if i == 0:  # Первый месяц
                period_start = start_date
            else:
                period_start = month_start

            if i == len(months) - 1:  # Последний месяц
                period_end = end_date
            else:
                period_end = month_start + pd.offsets.MonthEnd(0)

            # Считаем дни и продажи за месяц
            days_in_period = (period_end - period_start).days + 1
            month_sales = sales_per_day * days_in_period

            results.append({
                'project': name,
                'year': period_start.year,
                'months': period_start.month,
                'sales_live': month_sales,
                'developer': developer,
                'region': region,
                'okrug': okrug,
                'corpus': corpus,
                'ID': ID,
                'price':price
            })

        # Обработка последнего месяца, если он не был включен в цикл
        if end_date > (months[-1] + pd.offsets.MonthEnd(0)):
            period_start = months[-1] + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1)
            period_end = end_date
            days_in_period = (period_end - period_start).days + 1
            month_sales = sales_per_day * days_in_period

            results.append({
                'project': name,
                'year': period_start.year,
                'months': period_start.month,
                'sales_live': month_sales,
                'developer': developer,
                'region': region,
                'okrug': okrug,
                'corpus': corpus,
                'ID': ID,
                'price':price
            })

    # Создаем и группируем результирующий датафрейм
    result_df = pd.DataFrame(results)
    result_df = result_df.groupby(['project', 'year', 'months', 'developer', 'region', 'okrug', 'corpus', 'ID', 'price'])['sales_live'].sum().reset_index()

    # Сортируем результаты
    result_df = result_df.sort_values(['project', 'year', 'months'])
    return result_df